# Atelier théorique #7 - **Sélection des variables**

### Objectifs de cette leçon
1. Apprivoiser les méthodes automatiques de sélection de variables.
2. Comparer différentes méthodes de sélection.
3. Présenter des résultats en tableau Pandas.

### 3 techniques montrées avec Scikit-learn
1. Sélection univariée des variables
2. Élimination récursive des variables
3. Importance des variables

### Charger les données et ajouter les titres de colonnes

In [ ]:
# Importer la librairie pandas
import pandas as pd

# Télécharger le jeu de données
data = pd.read_csv('diabetes.csv')

# Remplacer le nom des colonnes
colnames = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data.columns = colnames

data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Importer les librairies Python utilisées dans le Notebook

In [ ]:
# Importer les librairies Python
import numpy as np
import pandas as pd
np.set_printoptions(precision=2)

# Sélection univariée
from sklearn.feature_selection import SelectKBest, chi2, f_classif

# Recursive feature elimination (RFE)
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Importance des variables
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier

## 1. Sélection univariée

**Description** : Avec cette méthode, on sélectionne les variables qui expliquent la plus grande variabilité de la variable réponse/cible à l'aide d'une fonction de score pré-définie.

**Contexte d'utilisation** : 
* La fonction `SelectKBest` va retirer toutes les variables sauf celles qui ont les plus hauts scores (défini par l'argument `k`).
* Les scores sont déterminés par l'argument de fonction sélectionné. Il y a plusieurs choix disponibles selon le type de données avec lesquels on travaille.

**Choix de l'argument fonction de score**

La fonction de score correspond au test statistique qui est utilisé pour classer les variables.

* Pour une régression : `f_regression`
* Pour une classification : `f_classif` (*par défaut*), `chi2` (*pour des variables dont les valeurs sont positives seulement*)

**Liens** : 
* Guide d'utilisation : https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection
* `SelectKBest` : https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

In [ ]:
# Importer les librairies nécessaires
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [ ]:
data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Séparer les données en sous-ensemble X et y



In [ ]:
# Préparation des données
X = data.iloc[:, 0:8]
y = data.iloc[:, 8]

In [ ]:
# Imprimer l'aperçu des colonnes incluses dans X
X.head()

,preg,plas,pres,skin,test,mass,pedi,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [ ]:
# Imprimer l'aperçu de le colonne incluse dans y
y.head()

0    1
1    0
2    1
3    0
4    1
Name: class, dtype: int64

### Avec l'argument de fonction chi2

In [ ]:
# Sélection des variables avec les 4 plus hauts scores
test_chi2 = SelectKBest(score_func=chi2, k=4)
fit_chi2 = test_chi2.fit(X, y)

In [ ]:
# Imprimer les scores
np.set_printoptions(precision=2)
print(f'Score des variables pour le chi2 : {fit_chi2.scores_}')

Score des variables pour le chi2 : [ 111.52 1411.89   17.61   53.11 2175.57  127.67    5.39  181.3 ]


In [ ]:
# Produire le tableau récapitulatif pour le KBest avec le chi2
kbest_chi2 = pd.DataFrame(data=[np.array(colnames[:-1]), fit_chi2.scores_, fit_chi2.get_support()]).T
kbest_chi2.columns = ['variable', 'score_kbest', 'support_kbest']
kbest_chi2['score_kbest'] = kbest_chi2['score_kbest'].astype(float)
kbest_chi2 = kbest_chi2.round({'score_kbest': 2})
kbest_chi2 = kbest_chi2.sort_values(by='score_kbest', ascending=False)
kbest_chi2

,variable,score_kbest,support_kbest
4,test,2175.57,True
1,plas,1411.89,True
7,age,181.30,True
5,mass,127.67,True
0,preg,111.52,False
3,skin,53.11,False
2,pres,17.61,False
6,pedi,5.39,False


### Avec l'argument de fonction f_classif

In [ ]:
# Sélection des variables avec les 4 plus hauts scores
test_fclass = SelectKBest(score_func=f_classif, k=4)
fit_fclass = test_fclass.fit(X, y)

In [ ]:
# Imprimer les scores
np.set_printoptions(precision=2)
print(f'Score des variables pour le f_classif : {fit_fclass.scores_}')

Score des variables pour le f_classif : [ 39.67 213.16   3.26   4.3   13.28  71.77  23.87  46.14]


In [ ]:
# Produire le tableau récapitulatif pour le KBest avec le f_classif
kbest_fclass = pd.DataFrame(data=[np.array(colnames[:-1]), fit_fclass.scores_, fit_fclass.get_support()]).T
kbest_fclass.columns = ['variable', 'score_kbest', 'support_kbest']
kbest_fclass['score_kbest'] = kbest_fclass['score_kbest'].astype(float)
kbest_fclass = kbest_fclass.round({'score_kbest': 2})
kbest_fclass = kbest_fclass.sort_values(by='score_kbest', ascending=False)
kbest_fclass

,variable,score_kbest,support_kbest
1,plas,213.16,True
5,mass,71.77,True
7,age,46.14,True
0,preg,39.67,True
6,pedi,23.87,False
4,test,13.28,False
3,skin,4.30,False
2,pres,3.26,False


## 2. Élimination récursive des variables (RFE - Recursive feature elimination)

**Description** : En se basant sur les coefficients obtenus pour chaque variable d'un algorithme en particulier (ex. LogisticRegression), la variable avec la plus faible importance sera éliminer. Cette étape sera répétée en boucle jusqu'à ce qu'on atteigne le nombre de variables que l'on veut sélectionner et garder dans notre modèle.

**Lien** : 
* `RFE` : https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

In [ ]:
# Importer les librairies nécessaires
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
# Préparation des données
X = data.iloc[:,0:8]
y = data.iloc[:,8]

In [ ]:
X.head()

,preg,plas,pres,skin,test,mass,pedi,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [ ]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: class, dtype: int64

In [ ]:
# Extraction des variables
model = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=model, n_features_to_select=4)
fit = rfe.fit(X, y)

In [ ]:
fit

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=1000,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=4, step=1, verbose=0)

In [ ]:
# Imprimer les attributs associés au RFE
print(f'Num Features: {fit.n_features_}')
print(f'Selected Features: {fit.support_}')
print(f'Feature Ranking: {fit.ranking_}')

Num Features: 4
Selected Features: [ True  True False False False  True  True False]
Feature Ranking: [1 1 3 5 4 1 1 2]


In [ ]:
rfe_results = pd.DataFrame(data=[np.array(colnames[:-1]), fit.support_, fit.ranking_]).T
rfe_results.columns = ['variable', 'support_rfe', 'ranking_rfe']
rfe_results = rfe_results.sort_values(by='ranking_rfe')
rfe_results

,variable,support_rfe,ranking_rfe
0,preg,True,1
1,plas,True,1
5,mass,True,1
6,pedi,True,1
7,age,False,2
2,pres,False,3
4,test,False,4
3,skin,False,5


## 3. Importance des variables

**Description :**
* Algorithme imbriqué qui permet de déterminé les variables importantes.
* Possibilité d'utiliser différents modèle algorithmes et de les comparer par la suite.
* Tout algorithme ayant l'attribut `feature_importances_` peut être utilisé.

> *Note : Le jeu de données qu'on utilise est présente un problème de **classification** ce qui signifie que les algorithmes choisis doivent eux aussi s'appliquer à un problème de classification.* 

**Liens** : 
* RandomForestClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* ExtraTreeClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html 
* GradientBoostingClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier
* SelectFromModel : https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html 

In [ ]:
# Importer les librairies nécessaires
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier

In [ ]:
# Préparation des données
X = data.iloc[:,0:8]
y = data.iloc[:,8]

In [ ]:
X.head()

,preg,plas,pres,skin,test,mass,pedi,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [ ]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: class, dtype: int64

### Modèle 1 : Random Forest

In [ ]:
# Préparer l'algorithme Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X, y)

#  Extraire les scores d'importance pour chaque variable
rf_score = model_rf.feature_importances_
print(f'Importance Scores Random Forest: {rf_score}')

Importance Scores Random Forest: [0.08 0.26 0.09 0.07 0.07 0.17 0.12 0.14]


In [ ]:
# Préparer l'algorithme Random Forest
model_rf_support = SelectFromModel(RandomForestClassifier(), max_features=4)
model_rf_support.fit(X, y)

#  Extraire le support pour chaque variable
rf_support = model_rf_support.get_support()
print(f'Support Random Forest: {rf_support}')

Support Random Forest: [False  True False False False  True False  True]


In [ ]:
# Résumer les résultats dans un tableau
gb_results = pd.DataFrame(data=[np.array(colnames[:-1]), rf_score, rf_support]).T
gb_results.columns = ['variable', 'rf_score', 'rf_support']
gb_results['rf_score'] = gb_results['rf_score'].astype(float)
gb_results = gb_results.sort_values(by='rf_score', ascending=False)
gb_results

,variable,rf_score,rf_support
1,plas,0.262116,True
5,mass,0.166467,True
7,age,0.140830,True
6,pedi,0.122169,False
2,pres,0.088397,False
0,preg,0.082566,False
4,test,0.071275,False
3,skin,0.066179,False


### Modèle 2 : Extra Trees

In [ ]:
# Préparer l'algorithme Extra Trees
model_et = ExtraTreesClassifier()
model_et.fit(X, Y)

#  Extraire les scores d'importance pour chaque variable
et_score = model_et.feature_importances_
print(f'Importance Scores Extra Trees: {et_score}')

Importance Scores Extra Trees: [0.11 0.23 0.1  0.08 0.07 0.14 0.12 0.14]


In [ ]:
# Préparer l'algorithme Extra Trees
model_et_support = SelectFromModel(ExtraTreesClassifier(), max_features=4)
model_et_support.fit(X, Y)

#  Extraire le support pour chaque variable
et_support = model_et_support.get_support()
print(f'Support Extra Trees: {et_support}')

Support Extra Trees: [False  True False False False  True False  True]


In [ ]:
# Résumer les résultats dans un tableau
gb_results = pd.DataFrame(data=[np.array(colnames[:-1]), et_score, et_support]).T
gb_results.columns = ['variable', 'et_score', 'et_support']
gb_results['et_score'] = gb_results['et_score'].astype(float)
gb_results = gb_results.sort_values(by='et_score', ascending=False)
gb_results

,variable,et_score,et_support
1,plas,0.232654,True
7,age,0.142731,True
5,mass,0.141874,True
6,pedi,0.117932,False
0,preg,0.108008,False
2,pres,0.099139,False
3,skin,0.083598,False
4,test,0.074064,False


### Modèle 3 : Gradient Boosting

In [ ]:
# Préparer l'algorithme Gradient Boosting
model_gb = GradientBoostingClassifier()
model_gb.fit(X, Y)

#  Extraire les scores d'importance pour chaque variable
gb_score = model_gb.feature_importances_
print(f'Importance Scores Gradient Boosting: {gb_score}')

Importance Scores Gradient Boosting: [0.05 0.41 0.04 0.02 0.05 0.19 0.1  0.14]


In [ ]:
# Préparer l'algorithme Gradient Boosting
model_gb_support = SelectFromModel(GradientBoostingClassifier(), max_features=4)
model_gb_support.fit(X, Y)

#  Extraire le support pour chaque variable
gb_support = model_gb_support.get_support()
print(f'Support Gradient Boosting: {gb_support}')

Support Gradient Boosting: [False  True False False False  True False  True]


In [ ]:
# Résumer les résultats dans un tableau
gb_results = pd.DataFrame(data=[np.array(colnames[:-1]), gb_score, gb_support]).T
gb_results.columns = ['variable', 'gb_score', 'gb_support']
gb_results['gb_score'] = gb_results['gb_score'].astype(float)
gb_results = gb_results.sort_values(by='gb_score', ascending=False)
gb_results

,variable,gb_score,gb_support
1,plas,0.409443,True
5,mass,0.193633,True
7,age,0.136488,True
6,pedi,0.104932,False
0,preg,0.054102,False
4,test,0.046646,False
2,pres,0.036585,False
3,skin,0.018171,False


### Résumer les résultats dans un tableau 

In [ ]:
# Créer un tableau de résultat
feat_importance_results = pd.DataFrame(data=[np.array(colnames[:-1]),
                                rf_score,
                                rf_support,
                                et_score, 
                                et_support,
                                gb_score,
                                gb_support]).T

# Donner les noms de colonnes au tableau de résultats
feat_importance_results.columns = ['variable', 'rf_score', 'rf_support', 'et_score', 'et_support', 'gb_score', 'gb_support']

# Redéfinir les types de données numériques à float
feat_importance_results['rf_score'] = feat_importance_results['rf_score'].astype(float)
feat_importance_results['et_score'] = feat_importance_results['et_score'].astype(float)
feat_importance_results['gb_score'] = feat_importance_results['gb_score'].astype(float)

# Arrondir les scores à deux décimales après la virgule
feat_importance_results = feat_importance_results.round({'rf_score': 2, 'et_score': 2, 'gb_score': 2})

# Imprimer le tableau de résultats
feat_importance_results

,variable,rf_score,rf_support,et_score,et_support,gb_score,gb_support
0,preg,0.08,False,0.11,False,0.05,False
1,plas,0.26,True,0.23,True,0.41,True
2,pres,0.09,False,0.10,False,0.04,False
3,skin,0.07,False,0.08,False,0.02,False
4,test,0.07,False,0.07,False,0.05,False
5,mass,0.17,True,0.14,True,0.19,True
6,pedi,0.12,False,0.12,False,0.10,False
7,age,0.14,True,0.14,True,0.14,True


## **LEÇON #7 - EXERCICE CONTINU**

## Sélection des variables

> *Utiliser votre notebook individuel `notebook_pratique_classification_binaire.ipynb`*

### Description
En utilisant le notebook pratique que vous avez commencer à utiliser durant la dernière leçon, accomplir les tâches énumérées ci-dessous.

> **Répondre aux questions dans une cellule de texte Markdown au fur et à mesure que vous progressez.**

> **Refaites rouler le code en entier avant de commencer cet exercice.** Utiliser le menu Exécution/Tout exécuter pour y parvenir.

## Tâches à accomplir :
1. Effectuer une sélection univariée de variables en utilisant `SelectKBest`. 
  * **Question 1.1** : Identifier les 2 variables avec les plus hauts scores.
  * **Question 1.2** : Produire le tableau récapitulatif.
2. Effectuer une élimination récursive des variables (RFE).
  * **Question 2.1** : Identifier les 2 variables avec les rangs les plus élevés.
  * **Question 2.2** : Produire le tableau récapitulatif.
3. À l'aide d'un `RandomForestClassifier`, ...
  * **Question 3.1** : Identifier les deux variables avec les scores d'importance les plus élevés. 
  * **Question 3.2** : Sont-elles supportées par l'algorithme RandomForest?
  * **Question 3.3** : Produire le tableau récapitulatif.
4. **Question 4**: En vous basant sur l'ensemble des résultats de sélection de variables obtenus, dans quel ordre d'importance classeriez-vous chaque variable?
  * **Question 4.1** : Y a-t-il une variable qui semble **meilleure** que toutes les autres? Si oui, laquelle?
  * **Question 4.2** : Y a-t-il une variable qui semble **pire** que toutes les autres? Si oui, laquelle?